In [6]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
from barcode import Code128
from barcode.writer import ImageWriter
from io import BytesIO
import pandas as pd
import os
import pandas as pd


def create_id_card(name, job_title, id_num, email, mobile, photo_path, barcode_output_path, id_card_output_path,validity):
    template_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\Temp ID Students.jpg"
    template = Image.open(template_path)
    draw = ImageDraw.Draw(template)
    font_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\raleway\\Raleway-ExtraBold.ttf"

    # Name and job title
    Name_font = ImageFont.truetype(font_path, size=48)
    job_title_font = ImageFont.truetype(font_path, size=28)

    name_bbox = draw.textbbox((0, 0), f"{name}", font=Name_font)
    x_name = (template.width - name_bbox[2]) // 2

    job_title_bbox = draw.textbbox((0, 0), f"{job_title}", font=job_title_font)
    x_job = (template.width - job_title_bbox[2]) // 2

    y_coordinate = 690

    draw.text((x_name, y_coordinate), f"{name}", font=Name_font, fill="black")
    draw.text((x_job, y_coordinate + 52), f"{job_title}", font=job_title_font, fill="#ABB0B8")

    # Other details
    font_path3="C:/Users/GALDIN/Desktop/Data Science/Project/ID card/liberation-serif-font/LiberationSerifBold-vg1L.ttf"
    detail_font = ImageFont.truetype(font_path3, size=27)

    id_num_font = ImageFont.truetype(font_path3, size=29)
    email_font = ImageFont.truetype(font_path3, size=25)
    mobile_font = ImageFont.truetype(font_path3, size=28)

    
    x = 55
    y = 790

    draw.text((x, y), f"Badge ID :", font=detail_font, fill="grey")
    draw.text((x, y + 39), f"Email :", font=detail_font, fill="grey")
    draw.text((x, y + 75), f"Mob   :", font=detail_font, fill="grey")

    draw.text((x + 130, y), f"{id_num}", font=id_num_font, fill="grey")
    draw.text((x + 91, y + 39), f"{email}", font=email_font, fill="grey")
    draw.text((x + 90, y + 71), f"{mobile}", font=mobile_font, fill="grey")

    # Create a larger circular mask for the photo
    photo = Image.open(photo_path).convert("RGBA")
    mask_size = (photo.size[0] + 55, photo.size[1] + 55)
    mask = Image.new("L", mask_size, 0)
    draw_mask = ImageDraw.Draw(mask)
    draw_mask.ellipse((0, 0, mask_size[0], mask_size[1]), fill=255)

    # Apply the circular mask to the photo
    photo = ImageOps.fit(photo, mask.size, centering=(0.5, 0.5))
    photo.putalpha(mask)

    # Adjust the coordinates for pasting the circular photo based on your template layout
    template.paste(photo, (149, 290), photo)

    # Paste barcode onto the ID card
    barcode_image = generate_barcode(id_num, width=330, height=90)
    template.paste(barcode_image, (55, 902))
    
    valid_upto = ImageFont.truetype(font_path3, size=20)
    draw.text((121, 988), f"Valid to : {validity}", font=valid_upto, fill="black")

    # Save barcode image
    barcode_image.save(barcode_output_path)

    # Save ID card
    template.save(id_card_output_path)
    template.show()

    
    
    
    
def generate_barcode(id_num, width, height=80):
    barcode = Code128(f'{id_num}', writer=ImageWriter())
    buffer = BytesIO()
    barcode.write(buffer)
    barcode_image = Image.open(buffer)

    # Remove human-readable text by cropping and resizing the image 
    barcode_image = barcode_image.crop((0, 0, barcode_image.width, barcode_image.height-73))
    barcode_image = barcode_image.resize((width, height))

    return barcode_image





def create_id_cards_from_excel(excel_path, output_folder):
    df = pd.read_csv(excel_path)

    for index, row in df.iterrows():
        # Customize the output paths and file names based on your needs
        barcode_output_path = f"{output_folder}/Barcode/{row['Name']}.png"
        id_card_output_path = f"{output_folder}/{row['Name']}.jpg"

        create_id_card(
            name=row['Name'],
            job_title=row['Job Title'],
            id_num=row['ID'],
            email=row['Email'],
            mobile=row['Mobile'],
            photo_path=row['Photo Path'],
            barcode_output_path=barcode_output_path,
            id_card_output_path=id_card_output_path,
            validity=row['Validity']
        )

In [9]:
excel_path = "C:/Users/GALDIN/Desktop/Data Science/Project/ID card/Details.csv"
output_folder = "C:/Users/GALDIN/Desktop/Data Science/Project/ID card/ID cards"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

create_id_cards_from_excel(excel_path, output_folder)